<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Diabetes prediction using Decision Forest Classifier and GLM
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>According to research, there are a lot of people that are suffering from diabetes all over the world. Studies show that in 2019, diabetes was the direct cause of 1.5 million deaths and almost 50% of all deaths occurred before the age of 70.
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>  
Over time, diabetes can have a negative impact on multiple organs. It can damage the heart, blood vessels, eyes, kidneys, and nerves. The earlier a person receives a proper treatment, the more likely he or she will be in lowering blood glucose level. In addition, the risk of failure of other organs will also be reduced.
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>  
Hence, we are showcasing the complete approach about how we can make prediction of diagnosis of  diabetes 6 months in advance. We are demonstrating how In-Database Functions can be used for model training and scoring and comparing the performance of 2 models. The data we are using is a sample dataset and the results and predictions may not be entirely accurate.
</p>




<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b> Contents:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li  style = 'font-size:14px;font-family:Arial;color:#00233C'>Configuring the Environment</li>
    <li  style = 'font-size:14px;font-family:Arial;color:#00233C'>Initiate a connection to Vantage</li>
    <li  style = 'font-size:14px;font-family:Arial;color:#00233C'>Analyze the raw data set</li>
    <li  style = 'font-size:14px;font-family:Arial;color:#00233C'>Train and Test a Decision Forest Model</li>
        <ul  style = 'font-size:14px;font-family:Arial;color:#00233C'>
            <li  style = 'font-size:14px;font-family:Arial;color:#00233C'>4.1 Train and Test split using SAMPLE. Splitting the dataset in 80:20 ratio for Train and Test respectively</li>
            <li  style = 'font-size:14px;font-family:Arial;color:#00233C'>4.2 Train a Model</li> 
                <ol  style = 'font-size:14px;font-family:Arial;color:#00233C'>
                    <li style = 'font-size:14px;font-family:Arial;color:#00233C' >Using the DecisionForest and TDDecisonForestPredict In Database function to predict if the person can have Diabetes or not. So, there are only 2 responses '0' and '1'.</li>
                    <li style = 'font-size:14px;font-family:Arial;color:#00233C'>Using the GLM and TDGLMPredict In Database function to predict if the person can have Diabetes or not. So, there are only 2 responses '0' and '1'.</li>
            </ol>
            <li style = 'font-size:14px;font-family:Arial;color:#00233C'>4.3 Evaluate the Model :- Evaluation of the model is done using the TD_ClassificationEvaluator which provides various parameters for the model like Accuracy, Precision ,Recall etc.</li>
        </ul>
    <li style = 'font-size:14px;font-family:Arial;color:#00233C'>Cleanup</li>
</ol>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Business Values</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Comprehensive health predictions and a reduced number of false positive and false negative results.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Reduced cost to patients and hospitals caused by heart attacks.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Identify patterns and symptoms leading to heart failure to ensure early intervention.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Advanced research and development stemming from the results of the data and models produced.</li></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Machine Learning and AI have a proven track record of improving patient outcomes and well-being across the entire healthcare industry. Traditional approaches to data preparation, model development, and deployment rely on manual, error-prone processes that prevent enterprises from realizing the true value of these tools and techniques.</p>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>However, Vantage provides these same proven data preparation and machine learning capabilities, integrated as native ClearScape Analytic functions.  This allows organizations to drastically reduce data preparation, model development, and testing time, while allowing for much more frequent and iterative testing and tuning to ensure maximum life-critical accuracy. Furthermore, the exact same development pipeline can be deployed seamlessly to production, eliminating the traditional development-to-deployment gap in the ML and AI industry.</p>


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Connect to Vantage</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
# Import necessary libraries.
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
import json
import getpass
import pandas as pd
from teradataml import *

import teradatasql
import plotly.express as px
import plotly.figure_factory as ff
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import numpy as np
from sklearn import metrics
display.max_rows = 5

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=Diabetes_Classification_DFandGLM.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage.  Here we are downloading the data to local storage which may yield somewhat faster execution, but there could be considerations of available storage. We are only creating local databases and tables as there are 625 columns in table which will be faster in local tables as compared to foreign tables.</p> 
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>“Note:  The data loading part of this demo will be slow because we have large number of  columns (625 columns)."</b></p>    


In [ ]:
%run -i ../run_procedure.py "call get_data('DEMO_DiabetesPrediction_local');"
 # Takes about 3 minutes 30 seconds


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Analyze the raw data set</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us start by creating a "Virtual DataFrame" that points directly to the dataset in Vantage. We then begin our analysis by checking the shape of the DataFrame and examining the data types of all its columns.</p>



In [ ]:
df=DataFrame(in_schema("DEMO_DiabetesPrediction","Diabetes_Diagnosis_Data"))
df

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Exploratory Data Analysis</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>With the help of exploratory data analysis (EDA) techniques, we will summarize the main characteristics about the data.
</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can see that the aggregated data is available to us in teradataml dataframe. Let's visualize this data to better understand the distribution of the data by different parameters. Vantage's Clearscape Analytics can easily integrate with 3rd party visualization tools like Tableau, PowerBI or many python modules available like plotly, seaborn etc. We can do all the calculations and pre-processing on Vantge and pass only the necessary information to visualization tools, this will not only make the calculation faster but also reduce the time due to less data movement between tools. We do the data transfer for this and the subsequent visualizations wherever necessary.</p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.1. Age Distribution Plot</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Following plot shows the density distribution of the people that were diagnosed with diabetes or not based on their age.
</p>

In [ ]:
DF=df.to_pandas()
plt.rcParams['figure.figsize'] = [8, 2]
Target = {0:'Non-Diabetic',1:'Diabetic'}
for label in Target:
    X = DF[DF["target"] == label]
    sns.distplot(X["dem_age"], hist = False, kde = True,
                 kde_kws = {'shade': True, 'linewidth': 3}, 
                  label = Target[label])

plt.legend(prop={'size': 8}, title = 'Target')
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Density')
plt.show()

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.2. Gender Distribution Plot</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Following plot highlights the demographics showing the count of males and females.
</p>

In [ ]:
df_plot = df.select(['dem_female','MBR_ID']).groupby('dem_female').count()
df_plot_pd = df_plot.to_pandas()
df_plot_pd

In [ ]:
sns.barplot(x = 'dem_female', y = 'count_MBR_ID', data = df_plot_pd)
plt.xlabel('Gender')
plt.ylabel('Frequency')
plt.xticks(ticks = [0, 1], labels = ['Male', 'Female'])
plt.title('Gender Distribution')
plt.show()

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.3. Target Distribution Plot</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Following plot highlights the count of diabetic and non-diabetic patients.
</p>

In [ ]:
df_plot = df.select(['target','MBR_ID']).groupby('target').count()
df_plot_pd = df_plot.to_pandas()
df_plot_pd

In [ ]:
sns.barplot(x = 'target', y = 'count_MBR_ID', data = df_plot_pd)
plt.xlabel('Target')
plt.ylabel('Frequency')
plt.xticks(ticks = [0, 1], labels = ['Non-Diabetic', 'Diabetic'])
plt.title('Target Distribution')
plt.show()

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>4.4. SNOMED Codes Distribution Plot</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Following plot highlights how many visits consist of diagnoses, procedures and medications.
</p>

In [ ]:
Diag = df.filter(regex="^diag").sum().get_values().sum()
Proc = df.filter(regex="^proc").sum().get_values().sum()
Drg = df.filter(regex="^drg").sum().get_values().sum()

In [ ]:
Dict = {"Diagnosis":Diag,"Procedures":Proc,"Drugs":Drg}
plt.bar(list(Dict.keys()),Dict.values())
plt.xlabel("Medication")
plt.ylabel("Frequency")
plt.title("SNOMED Codes Distribution")
plt.show()

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Feature Selection</b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Currently we have 622 features excluding ID and target. We used regression-based feature reduction technique to eliminate uninformative features.
</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1. Create Train and Test Dataset</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now that we have our prepared data set, we can perform an abbreviated machine learning workflow:</p>

<ol style = 'font-size:16px;font-family:Arial'>
    <li>Create Train and Test data sets using SAMPLE Clause(80:20 split)</li>
    <li>Train the model</li>
    <li>Evaluate the model using Test data</li>
</ol>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Train and Test split using SAMPLE</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use EXCEPT clause in the second statement to ensure a non-intersecting set of data</p>

In [ ]:
diabetes_features_train = df[df.dem_age.mod(10) >=1]
df_train_count = diabetes_features_train.select(['target','MBR_ID']).groupby('target').count()
df_train_count

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The output shows the number of people we are considering for each class to train the model – class 1 has Diabetes</p>

In [ ]:
diabetes_features_test = df[df.dem_age.mod(10) < 1]
df_test_count = diabetes_features_test.select(['target','MBR_ID']).groupby('target').count()
df_test_count

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The output shows the number of people we are considering for each class to validate the model – class 1 has Diabetes</p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2. Using GLM with ElasticNet regularization for feature reduction.</b></p>

In [ ]:
GLM_out = GLM(input_columns= ['2:622'],
                    response_column = "target",
                    data = diabetes_features_train,
                    family='Binomial',
                    learning_rate = 'optimal',
                    batch_size=200,
                    max_iter_num=100,
                    alpha=0.15,
                    lambda1=0.00175,
                    iter_num_no_change=50,
                    tolerance=0.0001,
                    intercept=True,
                    initial_eta=0.001,
                    momentum = 0.0,
                    local_sgd_iterations=0
                    )
output=GLM_out.result

In [ ]:
output[output["estimate"] != 0].sort("attribute")

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>7.3. Storing the selected features in a new table.</b></p>

In [ ]:
list_attr = list(output[output["estimate"] == 0].select(["attribute"]).get_values()[:,0])
Total = list(range(2,623))
dif1 = np.setdiff1d(Total, list_attr)
dif2 = np.setdiff1d(list_attr, Total)
temp3 = np.concatenate((dif1, dif2))
Features = list(temp3)
Features = list(filter(lambda x : x > 0, Features))
Str = ""
for i in range(len(Features)):
    Str += "{}".format(diabetes_features_train.columns[Features[i]])
    if i != (len(Features) - 1):
        Str += ","
Str        
final_list =  "MBR_ID" +"," + "target"  + "," + Str 
column_list = final_list.split(',')

In [ ]:
train_dataset = diabetes_features_train[column_list]
test_dataset = diabetes_features_test[column_list]

In [ ]:
df_train_count = train_dataset.select(['target','MBR_ID']).groupby('target').count()
df_train_count

In [ ]:
df_test_count = test_dataset.select(['target','MBR_ID']).groupby('target').count()
df_test_count

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>8. Decision Tree Model</b></p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>8.1 - Train a Decision Tree Model</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-TRAINING-functions/DecisionForest'>DecisionForest</a> is an ensemble algorithm used for classification and regression predictive modeling problems. It is an extension of bootstrap aggregation (bagging) of decision trees. </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function takes the training data as input, as well as the following function parameters</p>
    <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>InputColumns; list or range of columns used as features (we used an ordinal reference of columns 2:217)</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>ResponseColumn; the dependent or target value (we used “class”, the first column)</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>TreeType; either CLASSIFICATION or REGRESSION</li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Other hyperparameter values detailed in the documentation</li>
        </ul>

In [ ]:
DecisionForest_out = DecisionForest(data = train_dataset, 
                            input_columns = ['2:217'], 
                            response_column = 'target', 
                            max_depth = 16, 
                            num_trees = 8, 
                            min_node_size = 1, 
                            mtry = -1, 
                            mtry_seed = 3,
                            seed = 3, 
                            tree_type = 'CLASSIFICATION')

decision_df=DecisionForest_out.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The DecisionForest function produces a model and a JSON representation of the decision tree. Below is explaination for some columns in the JSON tree. The other details can be found at the link <a href = 'https://docs.teradata.com/search/all?query=TD_DecisionForest&content-lang=en-US'>here.</a></p>

</p>
<html>
   <head>
      <style>
         table, th, td {
            border: 1px solid black;
            border-collapse:collapse;
         }
      </style>
   </head>
   <body>
      <table>
         <tr>
            <th>JSON Type</th>
            <th>Description</th>             
         </tr>
         <tr>
            <td>id_</td>
            <td>"Node identifier"</td>
         </tr>
         <tr>
            <td>nodeType_</td> 
            <td>The node type. Possible values: CLASSIFICATION_NODE,CLASSIFICATION_LEAF,REGRESSION_NODE,REGRESSION_LEAF.</td>
         </tr>
         <tr>
            <td>split_</td> 
            <td>The start of JSON item that describes a split in the node.</td>
         </tr> 
         <tr>
            <td>responseCounts_</td> 
            <td>[Classification trees] Number of observations in each class at node identified by id.</td>
         </tr>
         <tr>
            <td>size_</td> 
            <td>Total number of observations at node identified by id.</td>
         </tr> 
         <tr>
            <td>maxDepth_</td> 
            <td>Maximum possible depth of tree, starting from node identified by id. For root node, the
value is max_depth. For leaf nodes, the value is 0. For other nodes, the value is the
maximum possible depth of tree, starting from that node.</td>
         </tr>  
      </table>
   </body>
</html>


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>8.2 - Evaluate the Model</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Execute a testing prediction using the split data above.  Evaluate the model by creating a confusion matrix with the <a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-EVALUATION-functions/ClassificationEvaluator'>ClassificationEvaluator</a> SQL Function.</p>


<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Execute <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Model-Scoring-Functions/DecisionForestPredict'>TDDecisionForestPredict</a> using the model built above</li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Execute <a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-EVALUATION-functions/ClassificationEvaluator'>ClassificationEvaluator</a> and pass the actual classification and the predicted value</li>
</ol>

In [ ]:
DF_Predict_out = TDDecisionForestPredict(
    newdata=test_dataset,
    object=DecisionForest_out.result,
    id_column='MBR_ID',
    accumulate='target',
    output_prob=True,
    output_responses=['0','1']
    )

DF_Predict_out.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The TDDecisionForestPredict function creates probabilities for the prediction made depending on the class and the Id columns. The output of the predict function is passed to the Classification Evaluator to get the parameters of the functions.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>DecisionForestPredict outputs the probability that each observation is in the predicted class. To use DecisionForestPredict output as input to ML Engine ROC function, you must first transform it to show the probability that each observation is in the positive class. One way to do this is to change the probability to (1- current probability) when the predicted class is negative. The prediction algorithm compares floating-point numbers. Due to possible inherent data type differences between ML Engine and Analytics Database executions, predictions can differ.</p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We create the <b>Confusion Matrix</b> to compare the actual and the Predicted values. Confusion matrix is a very popular measure used while solving classification problems. It can be applied to binary classification as well as for multiclass classification problems. Confusion matrices represent counts from predicted and actual values. It is an N x N matrix used for evaluating the performance of a classification model, where N is the number of target classes.</p>


In [ ]:
df=DF_Predict_out.result.to_pandas()
plt.rcParams['figure.figsize'] = [6, 6]
cm = confusion_matrix(df['target'], df['prediction'])
cmd = ConfusionMatrixDisplay(cm, display_labels=['DoesNotHaveDiabetes', 'HasDiabetes'])
cmd.plot()
plt.show()

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>8.3 - Use classification Evaluator for DecisionForestPredict</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will evaluate the model by creating a confusion matrix with the <a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-EVALUATION-functions/ClassificationEvaluator'>ClassificationEvaluator</a> Function.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In classification problems, a confusion matrix is used to visualize the performance of a classifier. The confusion matrix contains predicted labels represented across the row-axis and actual labels represented
across the column-axis. Each cell in the confusion matrix corresponds to the count of occurrences of labels
in the test data.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Apart from accuracy, the secondary output table returns micro, macro, and weighted-average metrics of precision, recall, and F1-score values.</p>


In [ ]:
ClassificationEvaluator_obj = ClassificationEvaluator(data=DF_Predict_out.result,
                                                          observation_column='target',
                                                          prediction_column='prediction',
                                                          labels=['0', '1'])
classeval_decisiondf = ClassificationEvaluator_obj.output_data
classeval_decisiondf

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above output has the secondary output table that returns micro, macro, and weighted-average metrics of precision, recall, and F1-score values.</p>
<table style = 'font-size:16px;font-family:Arial;color:#00233C'>
  <tr>
    <th>Column</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>Precision</td>
    <td>The positive predictive value. Refers to the fraction of relevant instances among
the total retrieved instances.</td>
  </tr>
  <tr>
    <td>Recall</td>
    <td>Refers to the fraction of relevant instances retrieved over the total amount of
relevant instances.</td>
  </tr>
  <tr>
    <td>F1</td>
    <td>F1 score, defined as the harmonic mean of the precision and recall.</td>
  </tr>
  <tr>
    <td>Support</td>
    <td>The number of times a label displays in the ObservationColumn.</td>
  </tr>
</table>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>8.4 - Show AUC-ROC Curve for DecisionForestPredict</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <a href = 'https://docs.teradata.com/search/all?query=TD_ROC&content-lang=en-US'>ROC</a> curve shows the performance of a binary classification model as its discrimination threshold varies. For a range of thresholds, the curve plots the true positive rate against false-positive rate.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function accepts a set of prediction-actual pairs as input and calculates the following values for a range of discrimination thresholds.</p>
    <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>True-positive rate (TPR)</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>False-positive rate (FPR)</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>The area under the ROC curve (AUC)</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Gini coefficient</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Other details are mentioned in the documentation</li>
    </ul>



In [ ]:
roc_df = ROC(data = DF_Predict_out.result, 
                    probability_column = "prob_1",
                    observation_column = "target",
                    positive_class="1"
                    )
roc_df.output_data

In [ ]:
auc = roc_df.result.get_values()[0][0]
auc

In [ ]:
plot_roc_df = roc_df.output_data
plot =  plot_roc_df.plot(x=plot_roc_df.fpr, y=plot_roc_df.tpr,
                         title="Receiver Operating Characteristic (ROC) Curve",
                         xlabel='False Positive Rate', 
                         ylabel='True Positive Rate', 
                         color="blue",
                         legend=f'AUC = {round(auc, 4)}',
                         legend_style='lower right',
                         grid_linestyle='--',
                         grid_linewidth=0.5)
 
# Display the plot.
plot.show()

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>9. Generalized Linear Model(GLM)</b></p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>9.1 - Train a GLM Model</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <a href = 'https://docs.teradata.com/search/all?query=TD_GLM&content-lang=en-US'>Generalized Linear Model (GLM)</a> is an extension of the linear regression model that enables the linear equation to relate to the dependent variables by a link function. The GLM function supports several distribution families and associated link functions. </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function takes the training data as input, as well as the following function parameters</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li>InputColumns; list or range of columns used as features (we used an ordinal reference of columns 2:217)</li>
        <li>ResponseColumn; the dependent or target value (we used “class”, the first column) </li>
        <li>Family; either Binomial or Gaussian</li>
    <li>Other hyperparameter values detailed in the documentation</li>
        </ul>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the GLM function to train the model on our data.</p>

In [ ]:
GLM_df = GLM(input_columns= ['2:217'],
                    response_column = "target",
                    data = train_dataset,
                    family='Binomial',
                    learning_rate = 'optimal',
                    batch_size=200,
                    max_iter_num=300,
                    alpha=0.15,
                    lambda1=0.01,
                    iter_num_no_change=50,
                    tolerance=0.008,
                    intercept=True,
                    initial_eta=0.05,
                    momentum = 0.8,
                    local_sgd_iterations=0
                    )
GLM_df.result

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The GLM function creates various output predictors and values based on the above parameters passed in the query</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The function output is a trained GLM model which can be input to the TDGLMPredict function
for prediction. The model also contains model statistics of MSE, Loglikelihood, AIC, and BIC.
Further model evaluation can be done as a post-processing step using functions such as
TD_RegressionEvaluator, TD_ClassificationEvaluator and TD_ROC.</p>


<p style = 'font-size:16px;font-family:Arial;> The TD_DecisionForest function creates a tree as seen in the output above based on the parameters applied in the query. </b></p>



<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>9.2 - Evaluate the Model</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Execute a testing prediction using the split data above.  Evaluate the model by creating a confusion matrix with the <a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-EVALUATION-functions/ClassificationEvaluator'>ClassificationEvaluator</a> Function.</p>


<ol style = 'font-size:16px;font-family:Arial'>
    <li>Execute <a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-SCORING-functions/TDGLMPredict'>GLMPredict</a> using the model built above</li>
    <li>Execute <a href = 'https://docs.teradata.com/r/Lake/Teradata-Package-for-Python-Function-Reference-on-VantageCloud-Lake/teradataml-Analytic-Database-17.20.xx-Analytic-Functions/MODEL-EVALUATION-functions/ClassificationEvaluator'>ClassificationEvaluator</a> and pass the actual classification and the predicted value</li>
</ol>

In [ ]:
TDGLMPredict_out = TDGLMPredict(object=GLM_df.result,
                                    newdata=test_dataset,
                                    accumulate="target",
                                    id_column="MBR_ID",
                                    output_prob=True,
                                    output_responses=['0','1'])
df=TDGLMPredict_out.result
df

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The TDGLMPredict function predicts target values (regression) and class labels (classification) for test data using a GLM model trained by the GLM function. Similar to GLM, input features should be standardized, such as using ScaleFit, and ScaleTransform, before using in the function. The function takes only numeric features. The categorical
features must be converted to numeric values prior to prediction.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Rows with missing (null) values are skipped by the function during prediction. For prediction results evaluation, you can use RegressionEvaluator, ClassificationEvaluator or ROC function as postprocessing step.</p>


<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>9.3 - Use classification Evaluator for GLMPredict</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Evaluate the model by creating a confusion matrix with the <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>ClassificationEvaluator</a> Function.</p>



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Since ClassificationEvaluator requires same datatype for prediction and class columns so creating another table with same datatype.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Create CONFUSION MATRIX for the GLM Predict model.</b></p>

In [ ]:
df_glm = df.to_pandas()
cm = confusion_matrix(df_glm['target'], df_glm['prediction']) # , normalize='all'
cmd = ConfusionMatrixDisplay(cm, display_labels=['DoesNotHaveDiabetes', 'HasDiabetes'])
cmd.plot()
plt.show()

In [ ]:
df_conv = df.assign(pred_int = df.prediction.cast(type_=INTEGER),
                    target_int = df.target.cast(type_=INTEGER))

In [ ]:
ClassificationEvaluator_glmobj = ClassificationEvaluator(data=df_conv,
                                                          observation_column='target_int',
                                                          prediction_column='pred_int',
                                                          labels=['0', '1'])
classeval_glmdf = ClassificationEvaluator_glmobj.output_data
classeval_glmdf

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>9.4 - Show AUC-ROC Curve for GLMPredict</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <a href = 'https://docs.teradata.com/search/all?query=TD_ROC&content-lang=en-US'>ROC</a> curve shows the performance of a binary classification model as its discrimination threshold varies. For a range of thresholds, the curve plots the true positive rate against false-positive rate.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function accepts a set of prediction-actual pairs as input and calculates the following values for a range of discrimination thresholds.</p>
    <ul style = 'font-size:16px;font-family:Arial'>
        <li>True-positive rate (TPR)</li>
        <li>False-positive rate (FPR)</li>
        <li>The area under the ROC curve (AUC)</li>
        <li>Gini coefficient</li>
        <li>Other details are mentioned in the documentation</li>
    </ul>



In [ ]:
roc_glmdf = ROC(data = df_conv, 
                    probability_column = "prob_1",
                    observation_column = "target",
                    positive_class="1"
                    )
roc_glmdf.output_data

In [ ]:
auc = roc_glmdf.result.get_values()[0][0]
auc

In [ ]:
plot_roc_df = roc_glmdf.output_data
plot =  plot_roc_df.plot(x=plot_roc_df.fpr, y=plot_roc_df.tpr,
                         title="Receiver Operating Characteristic (ROC) Curve",
                         xlabel='False Positive Rate', 
                         ylabel='True Positive Rate', 
                         color="blue",
                         legend=f'AUC = {round(auc, 4)}',
                         legend_style='lower right',
                         grid_linestyle='--',
                         grid_linewidth=0.5)
 
# Display the plot.
plot.show()

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>10. Comparison of the Metrics generated by the 2 Models. Decision Forest vs GLM</b></p>

In [ ]:
classeval_alldf = classeval_decisiondf.merge(right = classeval_glmdf, how='inner', on='SeqNum' , 
                                             lsuffix = 't1', rsuffix = 't2')
display.max_rows=10
classeval_alldf = classeval_alldf.assign(drop_columns=True,
                                         SeqNum = classeval_alldf.t1_SeqNum,
                                         Metric = classeval_alldf.t1_Metric,
                                         GLM_MetricValue = classeval_alldf.t2_MetricValue * 100,
                                         DF_MetricValue = classeval_alldf.t1_MetricValue * 100)
classeval_alldf.sort('SeqNum')

In [ ]:
figure = Figure(width=800, height=400, image_type="jpg",  heading="Comparison of Metric Values")
plot =  classeval_alldf.plot(x=classeval_alldf.SeqNum, 
                             
                             y=[classeval_alldf.DF_MetricValue, classeval_alldf.GLM_MetricValue],
                             color=['orange', 'blue'], figure=figure, kind='bar', 
                             xlabel = 'Metrics' , ylabel = 'Metric Values',
                            legend=['Decision Forest', 'GLM'])
 
# Display the plot.
plot.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Thus, here we have used 2 different models to train and predict the data. The classification evaluator is used to evaluate and compare the models. The Teradata In-Database functions are used for training, prediction and evaluation. In this case since we have sample data the result parameters like the Accuracy, Precision, Recall etc. may not be accurate for both the models, still from the above graph we can conclude that in this case DecisionForest is better than GLM. 
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data preparation and machine learning capabilities, integrated as native ClearScape Analytic functions in Vanatage allow organizations to drastically reduce data preparation, model development, and testing time, while allowing for much more frequent and iterative testing and tuning to ensure maximum life-critical accuracy. Furthermore, the exact same development pipeline can be deployed seamlessly to production, eliminating the traditional development-to-deployment gap in the ML and AI industry.</p>    


<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>11. Cleanup</b></p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will use the following code to clean up tables and databases created for this demonstration.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_DiabetesPrediction');" 
#Takes 40 seconds

In [ ]:
remove_context()


<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let’s look at the elements we have available for reference for this notebook:</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b> Data</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This dataset contains data for 10000 patients, half of which were diagnosed with diabetes. It contains 624 columns. We used patients' visit records consisting of diagnoses, procedures, medications and demographics. In addition, we also added a temporal aspect to the medical features. We differentiated between events occurring 1-3 months before diagnosis, 3-6 months, and 6-12 months, before the prediction window. The main aim of the data is to distinguish between healthy people and those who were diagnosed with diabetes, according to "target" column which is set to 0 for non-diabetic and 1 for diabetic.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://synthea.mitre.org/'>Link to the dataset</a>: This dataset was generated by Synthea for the experimentation purpose and does not reflect the actual population.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Filters:</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Industry:</b> Healthcare</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Functionality:</b> Machine Learning</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Use Case:</b> Prediction Analysis</li></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Related Resources:</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href='https://usc-word-edit.officeapps.live.com/we/%E2%80%A2%09https:/www.teradata.com/Blogs/Predicting-Heart-Failure-with-Teradata'>Saving Lives, Saving Costs: Predicting Heart Failure with Teradata</a></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href='https://www.teradata.com/Blogs/Hyper-scale-time-series-forecasting-done-right'>Hyper-scale time series forecasting done right</a></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href='https://www.teradata.com/Blogs/Forecasting-COVID-19-Using-Teradata-Vantage'>Forecasting COVID-19 Using Teradata Vantage</a>
    </li></p>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023,2024. All Rights Reserved
        </div>
    </div>
</footer>